In [1]:
from ollama import chat
import pandas as pd
import numpy as np
import ast
import json
from ollama import ChatResponse
from pydantic import BaseModel
from typing import List


In [2]:
sem_eval_root = 'SemEval_Task7'
sem_eval_test_root = 'SemEval_Task7_Test_Phase'

In [3]:
pairs_df = pd.read_csv(f'{sem_eval_root}/pairs.csv')
fact_checks_df = pd.read_csv(f'{sem_eval_root}/fact_checks.csv')
posts_df = pd.read_csv(f'{sem_eval_root}/posts.csv')
print(len(fact_checks_df))

153743


In [38]:
test_fact_checks_df = pd.read_csv(f'{sem_eval_test_root}/fact_checks.csv')
test_posts_df = pd.read_csv(f'{sem_eval_test_root}/posts.csv')


with open(f'{sem_eval_test_root}/crosslingual_predictions.json', 'r') as crosslingual_file:
  crosslingual_data = json.load(crosslingual_file)

crosslingual_prediction_post_id = list(map(lambda x: int(x), crosslingual_data))

,fact_check_id,claim,instances,title
0,0,"(' Are avocados good for you?', ' Are avocados...","[(1525653998.0, 'https://metafact.io/factcheck...",NaN
1,1,"(' Can animals have headaches?', ' Can animals...","[(1617955634.0, 'https://metafact.io/factcheck...",NaN
2,2,"("" Can we help prevent Alzheimer's with diet?""...","[(1525653998.0, 'https://metafact.io/factcheck...",NaN
3,3,(' Do any benefits of alcohol outweigh the ris...,"[(1525653998.0, 'https://metafact.io/factcheck...",NaN
4,4,"(' Does acupuncture work for headaches?', ' Do...","[(1617955595.0, 'https://metafact.io/factcheck...",NaN


In [5]:
def get_translated_claim(tuple_string):
  return ast.literal_eval(tuple_string)[1]

def get_translated_claim_lang(tuple_string):
  if isinstance(ast.literal_eval(tuple_string)[-1][0], str):
    return ast.literal_eval(tuple_string)[-1][0]
  else:
    return ast.literal_eval(tuple_string)[-1][0][0]

def get_translated_title(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  return ast.literal_eval(tuple_string)[1]


def get_translated_title_lang(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  if isinstance(ast.literal_eval(tuple_string)[-1][0], str):
    return ast.literal_eval(tuple_string)[-1][0]
  else:
    return ast.literal_eval(tuple_string)[-1][0][0]

def get_translated_ocr(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  eval = ast.literal_eval(tuple_string)
  if not eval:
    return ""
  return eval[0][1]

def get_translated_ocr_lang(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  eval = ast.literal_eval(tuple_string)
  if not eval:
    return ""
  return eval[0][-1][0][0]

def get_translated_text(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  eval = ast.literal_eval(tuple_string)
  if not eval:
    return ""
  return eval[1]

def get_translated_text_lang(tuple_string):
  if pd.isnull(tuple_string):
    return ""
  eval = ast.literal_eval(tuple_string)
  if not eval:
    return ""
  return eval[-1][0][0]

In [6]:
claim_translated = fact_checks_df['claim'].apply(get_translated_claim)
claim_lang = fact_checks_df['claim'].apply(get_translated_claim_lang)


fact_checks_df['claim_translated'] = claim_translated
fact_checks_df['claim_lang'] = claim_lang

In [7]:
title_translated = fact_checks_df['title'].apply(get_translated_title)
title_lang = fact_checks_df['title'].apply(get_translated_title_lang)

fact_checks_df['title_translated'] = title_translated
fact_checks_df['title_lang'] = title_lang

final_fact_checks_df = fact_checks_df.drop(columns=['claim', 'title', 'instances'])
final_fact_checks_df.tail()

,fact_check_id,claim_translated,claim_lang,title_translated,title_lang
153738,205744,"🇫🇷 In France, the military and civilian police...",ara,This video is not for the French police's acti...,ara
153739,205745,👆This little beautiful girl was seen in Mangal...,eng,A child rescued from a group of beggars in Man...,tam
153740,205747,"📌 Italians and foreigners, men and women, chil...",ara,These photos of a queue in front of a food aid...,ara
153741,205749,🔵Confirmed... Tomorrow the free messages will ...,por,WhatsApp will charge 0.37 cents per message st...,por
153742,205750,🕋🗃 WE OPEN THE BLACK BOX OF BNDES.......\n✅ DU...,por,Post mixes true data with incorrect numbers ab...,por


In [59]:
claim_translated = test_fact_checks_df['claim'].apply(get_translated_claim)
claim_lang = test_fact_checks_df['claim'].apply(get_translated_claim_lang)


test_fact_checks_df['claim_translated'] = claim_translated
test_fact_checks_df['claim_lang'] = claim_lang


title_translated = test_fact_checks_df['title'].apply(get_translated_title)
title_lang = test_fact_checks_df['title'].apply(get_translated_title_lang)

test_fact_checks_df['title_translated'] = title_translated
test_fact_checks_df['title_lang'] = title_lang

final_test_fact_checks_df = test_fact_checks_df.drop(columns=['claim', 'title', 'instances'])
final_test_fact_checks_df.tail()

,fact_check_id,claim_translated,claim_lang,title_translated,title_lang
272442,372883,Claudia Sheinbaum wants to change the Constitu...,spa,France Media Agency ...,fra
272443,372884,Israeli Ambassador raises poster commemorating...,por,France Media Agency ...,fra
272444,372891,The viral video shows African protesting at Qu...,eng,Fact Check: Viral Video Does NOT Show African ...,eng
272445,372893,Do you get notifications when you take a scree...,tur,Do you get notifications when you take a scree...,tur
272446,372897,Electric cars pollute more and catch fire more...,spa,They neither pollute more nor catch fire more ...,spa


In [9]:
ocr_translated = posts_df['ocr'].apply(get_translated_ocr)
ocr_lang = posts_df['ocr'].apply(get_translated_ocr_lang)
posts_df["ocr_translated"]  = ocr_translated
posts_df["ocr_lang"]  = ocr_lang

text_translated = posts_df['text'].apply(get_translated_text)
text_lang = posts_df['text'].apply(get_translated_text_lang)
posts_df["text_translated"]  = text_translated
posts_df["text_lang"]  = text_lang

final_posts_df = posts_df.drop(columns=["instances", "ocr", "verdicts", "text"])
final_posts_df.head()

,post_id,ocr_translated,ocr_lang,text_translated,text_lang
0,0,! Brazen vaccination fake by Markus Söder! It'...,deu,,
1,1,!! WARNING !! A new thing circulating now. Peo...,eng,,
2,2,"""Actually, he's a damn sight better than any o...",eng,,
3,3,"""Australia 50 MILLION doses of ""vacuna"" retira...",fra,,
4,4,"""Blessed are those persecuted by me cause ""The...",spa,,


In [10]:
ocr_translated = test_posts_df['ocr'].apply(get_translated_ocr)
ocr_lang = test_posts_df['ocr'].apply(get_translated_ocr_lang)
test_posts_df["ocr_translated"]  = ocr_translated
test_posts_df["ocr_lang"]  = ocr_lang

text_translated = test_posts_df['text'].apply(get_translated_text)
text_lang = test_posts_df['text'].apply(get_translated_text_lang)
test_posts_df["text_translated"]  = text_translated
test_posts_df["text_lang"]  = text_lang

final_test_posts_df = posts_df.drop(columns=["instances", "ocr", "verdicts", "text"])
final_test_posts_df.tail()

,post_id,ocr_translated,ocr_lang,text_translated,text_lang
24426,28085,,,🧐 The president separated... the constitution ...,spa
24427,28087,bruising runny ed 1 e Contents of the pack and...,eng,🧐🧐🧐,eng
24428,28089,,,"🧬Robert Malone, inventor of mRNA technology, s...",fra
24429,28090,Number of Covid-19 Deaths 4500 4000 3500 3000 ...,eng,🧵While you are distracted by the invasion of R...,por
24430,28091,Popularity and foot If you respond in a profes...,kor,"🧿 The president is an avatar... ""President, if...",kor


In [11]:
joined_with_post = pd.merge(pairs_df, final_posts_df, on='post_id', how='inner')
joined_with_post.head()

,post_id,fact_check_id,ocr_translated,ocr_lang,text_translated,text_lang
0,2228,33,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,
1,2228,23568,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,
2,2228,194577,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,
3,2229,33,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,
4,2229,23568,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,


In [12]:
final_df = pd.merge(joined_with_post, final_fact_checks_df, on='fact_check_id', how='inner')
final_df.head()

,post_id,fact_check_id,ocr_translated,ocr_lang,text_translated,text_lang,claim_translated,claim_lang,title_translated,title_lang
0,2228,33,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,,"""$4 trillion jobs plan"" unnecessary because 20...",eng,Posts on Biden jobs plan falsely claim 2020 un...,eng
1,2228,23568,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,,America had the lowest unemployment rate in hi...,eng,Fact check: Unemployment rate hit historic hig...,eng
2,2228,194577,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,,“A year ago we had the lowest unemployment in ...,eng,Unemployment was the lowest ever a year ago? N...,eng
3,2229,33,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,,"""$4 trillion jobs plan"" unnecessary because 20...",eng,Posts on Biden jobs plan falsely claim 2020 un...,eng
4,2229,23568,"WHY DO WE NEED A $4 TRILLION JOBS PLAN, WHEN A...",eng,,,America had the lowest unemployment rate in hi...,eng,Fact check: Unemployment rate hit historic hig...,eng


In [13]:
ocr_text = final_df['ocr_translated'].apply(lambda x: f"OCR Text: '{x}'")
text = final_df['text_translated'].apply(lambda x: f"Social Media Caption: '{x}'")
claim = final_df['claim_translated'].apply(lambda x: f"Output: '{x}'")

In [14]:
concat_col = ocr_text.str.cat(text, sep=', ').str.cat(claim, sep='\n ')
examples = concat_col[:20].str.cat(sep="\n\n\n")

In [15]:
class ClaimGeneratorResponse(BaseModel):
    claim: str

In [30]:
post_id_generated_claim = {}

i = 0
for post_id in crosslingual_prediction_post_id:
  ocr = test_posts_df[test_posts_df['post_id'] == post_id]["ocr_translated"].iloc[0]
  if pd.isna(ocr):
    ocr = ""

  text = test_posts_df[test_posts_df['post_id'] == post_id]["text_translated"].iloc[0]
  if pd.isna(text):
    text = ""

    

  cur_claim_prompt  = f"""
Task: Generate a concise and accurate claim made by a social media post.

Input:

    OCR Text: {ocr}
    Social Media Caption: {text}

Output:
JSON

{{
  "claim": [The claim made in the social media post, based on both the image text and the caption]
}}

Guidelines:

    The claim should be stated objectively and avoid any subjective language or interpretation.
    If the post makes multiple claims, focus on the most prominent or overarching one but also make sure other claims are also mentioned.
    If the post does not make a clear claim, but rather expresses an opinion or sentiment, rephrase it as a statement about the author's perspective.
    Ensure the claim is grammatically correct and free of spelling errors.

Example:

    OCR Text: "Study shows 97% of scientists agree climate change is real."
    Social Media Caption: "The science is clear! We need to act on climate change now."
    Output:

JSON

{{
  "claim": "97% of scientists agree climate change is real." 
}}
"""
  response: ChatResponse = chat(model='llama3.2:3b', messages=[
  {
    'role': 'user',
    'content': cur_claim_prompt,
  },
], format=ClaimGeneratorResponse.model_json_schema(),
options={"temperature": 0.4})
  claim = ClaimGeneratorResponse.model_validate_json(response.message.content)
  post_id_generated_claim[post_id] = claim.claim 
  print(f"{i+1} out of {len(crosslingual_prediction_post_id)}")
  i+= 1
 

# Write the dictionary to a JSON file
with open("claims.json", "w") as f:
    json.dump(post_id_generated_claim, f)

KeyError: 'ocr_translated'

In [32]:
with open(f'{sem_eval_test_root}/tasks.json', 'r') as crosslingual_file:
  crosslingual_data = json.load(crosslingual_file)

print(len(crosslingual_data["crosslingual"]["fact_checks"]))

272447


In [35]:
with open(f'claims.json', 'r') as claims_file:
  claims_data = json.load(claims_file)

claims_data

4000

In [46]:
check_claims = final_test_fact_checks_df["claim_translated"]
check_claims_id = final_test_fact_checks_df["fact_check_id"]
for claim_to_check in check_claims:
    print(claim_to_check)
    break

 Are avocados good for you?


In [76]:
class ClaimCompareResponse(BaseModel):
    same_claim: bool
    key_entities_in_claim_A: List[str]
    key_entities_in_claim_B: List[str]

In [88]:
j = 0


final_ans = {}
for post_id, claim in zip(claims_data.keys(), claims_data.values()):
    cur_list = []
    for fact_check_id, claim_to_check in zip(check_claims_id, check_claims):
        claim_check_prompt = f"""
Task: Compare Claim A and Claim B and find out whether they are the same claims.

Input:

    Claim A: {claim}
    Claim B: {claim_to_check}

Output:

Respond in JSON format with the following key-value pair:

    "same_claim": (true/false),
    "key_entities_in_claim_A": (A list of all the key entities in claim A),
    "key_enitities_in_claim_B": (A list of all the key entities in claim B)


Guidance:

To determine if the claims are the same, consider the following:

    1. Identify key entities and concepts: Extract the main entities (people, places, organizations, etc.) and concepts from each claim.
    2. Compare entities and concepts: Check if the same entities and concepts are present in both claims. If there are significant differences in the entities or concepts discussed, the claims are likely different.
    3. Consider paraphrasing and synonyms: Account for different phrasing and word choices. Claims can be the same even if they don't use the exact same words.
    4. Analyze sentence structure: Different sentence structures can convey the same meaning. Pay attention to the core intent of the claims.
    5. If the key entities in Claim A aren't present in Claim B or vice versa your output should immediately by False.
    
Examples:

Example 1:

    Claim A: The dog barked at the mailman.
    Claim B: The postal worker was barked at by the dog.
    Output:
    JSON

    {{
      "same_claim": true,
      "key_entities_in_claim_A": ["dog", "mailman"],
      "key_enitities_in_claim_B": ["dog", "postal worker"]
    }}

Example 2:

    Claim A: The sun rises in the east.
    Claim B: The sun sets in the west.
    Output:
    JSON

    {{
      "same_claim": false,
      "key_entities_in_claim_A": ["sun", "east"],
      "key_enitities_in_claim_B": ["sun", "west"],
    }}

Example 3:

    Claim A: Paris is the capital of France.
    Claim B: France's capital city is Paris.
    Output:
    JSON

    {{
      "same_claim": true,
      "key_entities_in_claim_A": ["Paris", "France"],,
      "key_enitities_in_claim_B": ["France", "Paris"],
    }}


"""
        response: ChatResponse = chat(model='llama3.2:3b', messages=[
  {
    'role': 'user',
    'content': claim_check_prompt,
  },
], format=ClaimCompareResponse.model_json_schema(),
options={"temperature": 0.4})
        claim_check_response = ClaimCompareResponse.model_validate_json(response.message.content)
        print("not same claim")
        print(claim_check_response.key_entities_in_claim_A)
        print(claim_check_response.key_entities_in_claim_B)
        if claim_check_response.same_claim:
            cur_list.append(fact_check_id)
            print("same claim! ====================================")
            print(claim)
            print(claim_to_check)
            print(claim_check_response.key_entities_in_claim_A)
            print(claim_check_response.key_entities_in_claim_B)
            print()
        print(f"{j+1} out of {len(claims_data.keys())*len(check_claims)}")
        j += 1

        
    final_ans[post_id] = cur_list

        

        
       
# Write the dictionary to a JSON file
with open("final.json", "w") as f1:
    json.dump(final_ans, f1)

not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['avocados']
1 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['animals', 'headaches']
2 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
["Alzheimer's", 'diet']
3 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['alcohol', 'risks']
4 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['acupuncture', 'headaches']
5 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['lack of sleep', 'headaches']
6 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['marijuana', 'brain']
7 out of 1089788000
not same claim
['Stingers', 'European', 'black market', 'Kharkov counteroffensive']
['Autism', 'prevalence']
8 

not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['Andalusia', 'youth unemployment', 'Spain']
56 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive', 'Biden', 'Social Security']
['Biden', 'pre-existing conditions', 'Social Security']
57 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive', 'Biden', 'Social Security']
['Biden', 'Social Security']
58 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive', 'Biden', 'Ron Klain']
['H1N1']
59 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['Burisma', 'John Kerry', 'U.S. taxpayer', 'Democratic senators']
60 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['Caema', 'contest']
61 out of 1089788000
not same claim
['European black market', 'Kh

not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['15 billion euros', 'racism', 'Marcelino da Mata']
109 out of 1089788000
not same claim
['European black market', 'Kharkov counteroffensive', 'Stingers weapons']
['Antrim County', 'Mich.', '15,000 people', '7,000 votes', 'voted', 'flipped']
110 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['@realDonaldTrump', 'Scaravan Stunt', '#Thanksgiving', '200 million dollars']
111 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive', 'Pfizer COVID-19 experimental jab']
['15-year-old boy', 'heart attack', 'COVID-19 experimental jab']
112 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive', 'Bernie']
['150 million people', 'gun manufacturers', 'liability', '2007', 'Bernie']
113 out of 1089788000
not same claim
['Stingers weapons', 'European bla

not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['Walmart', '(42 percent of) associates']
161 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['Joe', 'legislative proposals', 'President Trump']
162 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['43% of all FOOD Stamps', 'Illegals']
163 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['Californians', 'jury duty', 'citizens', 'registered voters']
164 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive', 'China']
['45 nations', 'travel restrictions', 'president']
165 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['FoxNews', '@FoxNews', '45.6 million people', '11.7 million people']
166 out of 1089788000
not same claim
['Stinger

not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['920 Women', 'Unborn Babies', 'Vaccinated']
214 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['920 women', 'unborn babies', 'vaccinated']
215 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['930 €', 'To prepare for life', '6,700 €', 'To screw our life']
216 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['world’s customers', 'beyond our shores']
217 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['scientists', 'dirt', 'malaria', 'Walter Reed', 'George Goethals']
218 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['coronavirus cases']
219 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov 

...

not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['Iranians', 'U.S.', 'Hillary Clinton', 'emails']
3272 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['people', 'Vaccine', 'Flu']
3273 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['people', 'vax']
3274 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['presidents', 'Supreme Court justice']
3275 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['pharmacists', 'states', 'physician prescriptions']
3276 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['Michigan', 'Ohio', 'Pennsylvania', 'North Carolina', 'South Carolina', 'Florida']
3277 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov 

not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['Gabriel', 'Cifuentes']
3323 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['Sergei Skripal', 'Yulia Skripal', 'Porton Down laboratory']
3324 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
["Ukrainian 'For taking Crimea' medals", 'Kherson region military committees', 'military registration and enlistment offices']
3325 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive', 'Trump', 'governors']
['Media', 'Trump', 'bipartisan governors']
3326 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['CDC', 'Medical Tyranny', 'Travelers', 'Masks']
3327 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['Medical-grade marijuana', 'patient', 'THC', '

not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive', 'Mike Coffman', 'Trump']
['Mike Coffman', 'Trump']
3374 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive', 'Mike Madigan', 'Kwame Raoul']
['property taxes']
3375 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive', 'Mike Madigan', 'Merry Marwig']
['Mike Madigan', 'Merry Marwig']
3376 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['Mike Pence', 'Indiana']
3377 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['millionaire', 'Chinese club', 'son', 'weight']
3378 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['GNR military', 'Covid-19']
3379 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
[

not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['Californians', 'clean water']
3429 out of 1089788000
not same claim
['European black market', 'Kharkov counteroffensive', 'Stingers weapons']
['Wisconsinites', 'Republicans', 'public schools']
3430 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['Planned Parenthood', 'rural areas', 'medically underserved areas']
3431 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['death row prisoners', 'people of color']
3432 out of 1089788000
not same claim
['Stingers weapons', 'European black market', 'Kharkov counteroffensive']
['older workers', 'retirement savings', 'zero']
3433 out of 1089788000
not same claim
['Stingers', 'European black market', 'Kharkov counteroffensive']
['Obama administration', 'United States', 'Central American Minors program', 'aliens']
3434 out of 1089788000
not same claim

KeyboardInterrupt: 